#### Change label to binary, Prevent Data leakage

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv("/Users/iseunghyeon/Desktop/4-1/MLops/aac_intakes_outcomes.csv")
df.head()

# Create binary classification target: Adopted (1), Others (0)
df['adopted'] = df['outcome_type'].apply(lambda x: 1 if x == 'Adoption' else 0)


# prevent Data leakage and remove unnecessary ID
drop_cols = ['outcome_type', 'outcome_subtype', 'animal_id_outcome', 'animal_id_intake',
             'outcome_datetime', 'outcome_monthyear', 'time_in_shelter']  

df = df.drop(columns=[col for col in drop_cols if col in df.columns])

# Separate features and labels
X = df.drop(columns='adopted')
y = df['adopted']

# 학습/테스트 데이터 분리 (예: 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


#### Check class ratio

In [36]:
# 클래스별 비율 확인
class_ratios = y.value_counts(normalize=True) * 100
print("\n📊 클래스별 비율 (%):\n", class_ratios.round(2))

import matplotlib.pyplot as plt

# 막대 그래프로 시각화
class_counts.plot(kind='bar', color='skyblue')
plt.title("Outcome Type 분포")
plt.xlabel("Outcome Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


📊 클래스별 비율 (%):
 adopted
0    57.83
1    42.17
Name: proportion, dtype: float64


/var/folders/bw/2mkdr3w938z4h0z84x6l4dmr0000gn/T/ipykernel_31196/1101474631.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


#### AutoML : MLJAR
- decision tree
- Random Forest
- Xgboost
- Catboost

In [40]:
from supervised.automl import AutoML

automl = AutoML(
    mode="Compete",                  
    eval_metric="f1",               
    total_time_limit=1000,           
    results_path="MLJAR_AAC",       
    algorithms=[                  
        "Decision Tree",
        "Random Forest",
        "Xgboost",
        "CatBoost"
    ]
)

automl.fit(X_train, y_train)

AutoML directory: MLJAR_AAC
The task is binary_classification with evaluation metric f1
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'mix_encoding', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree f1 0.808078 trained in 4.08 seconds
Disable stacking for split validation
* Step simple_algorithms will try to check up to 2 models
2_DecisionTree f1 0.810104 trained in 4.29 seconds
3_DecisionTree f1 0.810104 trained in 4.23 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost f1 0.869078 trained in 8.81 seconds
5_Default_CatBoost f1 0.878013 trained in 12.73 s

/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/supervised/preprocessing/label_binarizer.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/supervised/preprocessing/label_binarizer.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/supervised/preprocessing/label_binarizer.py:28: PerformanceWarning: DataFrame is highly fragmented.  This i

7_Xgboost_categorical_mix f1 0.869282 trained in 8.35 seconds
* Step golden_features will try to check up to 3 models
None 10
Add Golden Feature: time_in_shelter_days_multiply_outcome_hour
Add Golden Feature: time_in_shelter_days_sum_outcome_hour
Add Golden Feature: time_in_shelter_days_ratio_outcome_hour
Add Golden Feature: outcome_hour_ratio_time_in_shelter_days
Add Golden Feature: time_in_shelter_days_multiply_intake_number
Add Golden Feature: time_in_shelter_days_multiply_outcome_number
Add Golden Feature: outcome_number_ratio_time_in_shelter_days
Add Golden Feature: intake_number_ratio_time_in_shelter_days
Add Golden Feature: time_in_shelter_days_ratio_intake_number
Add Golden Feature: time_in_shelter_days_ratio_outcome_number
Created 10 Golden Features in 3.79 seconds.
18_CatBoost_GoldenFeatures f1 0.877007 trained in 17.02 seconds
23_CatBoost_GoldenFeatures f1 0.877792 trained in 17.76 seconds
19_CatBoost_GoldenFeatures f1 0.878217 trained in 11.42 seconds
* Step kmeans_features

/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


18_CatBoost_KMeansFeatures f1 0.876343 trained in 14.0 seconds


...

/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


50_CatBoost f1 0.879217 trained in 14.65 seconds


/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


51_CatBoost f1 0.877685 trained in 15.75 seconds
52_CatBoost f1 0.876892 trained in 15.46 seconds
53_Xgboost f1 0.871841 trained in 9.74 seconds
54_Xgboost f1 0.871925 trained in 8.23 seconds
55_Xgboost f1 0.871795 trained in 12.23 seconds
56_Xgboost f1 0.870997 trained in 7.15 seconds
57_RandomForest f1 0.836995 trained in 30.22 seconds
58_RandomForest f1 0.84544 trained in 19.21 seconds
59_RandomForest f1 0.840677 trained in 39.42 seconds
* Step ensemble will try to check up to 1 model
Ensemble f1 0.884176 trained in 6.26 seconds
AutoML fit time: 1033.58 seconds
AutoML best model: Ensemble


AutoML(algorithms=['Decision Tree', 'Random Forest', 'Xgboost', 'CatBoost'],
       eval_metric='f1', mode='Compete', results_path='MLJAR_AAC',
       total_time_limit=1000)

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 예측
preds_label = automl.predict(X_test)

# 평가 지표 계산
acc = accuracy_score(y_test, preds_label)
precision = precision_score(y_test, preds_label)
recall = recall_score(y_test, preds_label)
f1 = f1_score(y_test, preds_label)

print(f"Accuracy : {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")

Accuracy : 0.8850
Precision: 0.8401
Recall   : 0.8983
F1-score : 0.8682


/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/supervised/preprocessing/label_binarizer.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/supervised/preprocessing/label_binarizer.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/homebrew/Caskroom/miniforge/base/envs/mljar-env/lib/python3.8/site-packages/supervised/preprocessing/label_binarizer.py:28: PerformanceWarning: DataFrame is highly fragmented.  This i

In [46]:
!pip freeze > requirements.txt